# Explain pydantic

Pydantic is python library used for data validation.

What are the advantages of Pydantic ?
- **Powered by type hints** — with Pydantic, schema validation and serialization are controlled by type annotations; less to learn, less code to write, and seamless integration with your IDE and static analysis tools.
- **Speed** — Pydantic's core validation logic is written in Rust. As a result, Pydantic is among the fastest data validation libraries for Python.
- **JSON Schema** — Pydantic models can emit JSON Schema, allowing for easy integration with other tools.
- **Strict and Lax mode** — Pydantic can run in either strict mode (where data is not converted) or lax mode where Pydantic tries to coerce data to the correct type where appropriate.
- **Dataclasses, TypedDicts and more** — Pydantic supports validation of many standard library types including `dataclass` and `TypedDict`.
- **Customisation** — Pydantic allows custom validators and serializers to alter how data is processed in many powerful ways.
- **Ecosystem** — around 8,000 packages on PyPI use Pydantic, including massively popular libraries like FastAPI, huggingface, Django Ninja, SQLModel, & LangChain.
- **Battle tested** — Pydantic is downloaded over 360M times/month and is used by all FAANG companies and 20 of the 25 largest companies on NASDAQ. If you're trying to do something with Pydantic, someone else has probably already done it.

**Table of contents**:
1. [Pydantic class](#1-pydantic-examples)
   1. [Pydantic in general](#11-pydantic-in-general)
   2. [Fields](#12-fields)
   3. [Define a pydantic object](#13-define-a-pydantic-object)
   4. [Protect fields](#14-protect-fields)
2. [Validation](#2-validation)
   1. [Field validators](#21-field-validators)
   2. [Model validators](#22-model-validators)

# 1. Pydantic class

## 1.1 Pydantic in general

Pydantic is a good way to create a python class and validate its attributes.

One of the primary ways of defining schema in Pydantic is via models. `Models` are simply classes which inherit from `BaseModel` and define fields as annotated attributes. Models share many similarities with Python's dataclasses, but have been designed with some subtle-yet-important differences that streamline certain workflows related to validation, serialization, and JSON schema generation. You can find more discussion of this in the Dataclasses section of the docs.

Untrusted data can be passed to a model and, after parsing and validation, Pydantic guarantees that the fields of the resultant model instance will conform to the field types defined on the model.

In [1]:
from pydantic import BaseModel, ConfigDict


class User(BaseModel):
    id: int
    name: str = 'Jane Doe'

    model_config = ConfigDict(str_max_length=10)

## 1.2 Fields

Attributes of Pydantic class are defined as fields.

These fields can have different types. You can use the standard types library but also Strict types from the Pydantic library and custom data types:
- Standard Library Types: Types from the python standard library.
- Strict Types: Types that enable you to prevent coercion from compatible types.
- Custom Data Types: Create your own custom data types.
- Field Type Conversions: Strict and lax conversion between different field types.

In [2]:
from pydantic import BaseModel, ConfigDict, StrictBool, StrictInt, StrictStr


class User(BaseModel):
    id: StrictInt
    name: StrictStr = 'Jane Doe'
    is_active: StrictBool = True

    model_config = ConfigDict(str_max_length=10)

Fields can be customized in a number of ways using the `Field()` function. You can add a default value, a pattern for strings, description of the attribute, etc.

In [3]:
from pydantic import BaseModel, Field, ConfigDict, StrictInt, StrictStr

class User(BaseModel):
    id: StrictInt
    name: StrictStr = Field(
        default="Jane Doe",
        pattern=r"[A-Z]{1}[a-z]+ [A-Z]{1}[a-z]",
        description="The user's full name.",
    )

    model_config = ConfigDict(str_max_length=10)

## 1.3 Define a pydantic object

Let's define our first pydantic class. For this class, we will define one to store information about musicians.

In [4]:
from pydantic import BaseModel, Field, StrictBool, StrictStr, ValidationError

class Musician(BaseModel):
    """Class representing a musician with validated fields."""

    name: StrictStr = Field(
        pattern=r"[A-Z]{1}[a-z]+ [A-Z]{1}[a-z]",
        description="The name of the musician. First and last name, capitalized."
    )
    instrument: StrictStr = Field(
        pattern=r"(Guitar|Bass|Drums|Vocals|Keyboard)",
        description="The instrument played by the musician.",
    )
    genre: StrictStr = Field(
        pattern=r"[A-Za-z]+",
        description="The genre of music the musician is associated with."
    )
    years_active: int = Field(
        ge=0,
        le=100,
        description="Number of years the musician has been active in their career.",
    )
    is_active: StrictBool = Field(
        default=True,
        description="Indicates if the musician is currently active.",
    )

In [5]:
john_mayer = Musician(
    name="John Mayer",
    instrument="Guitar",
    genre="Rock",
    years_active=27,
)

john_mayer

Musician(name='John Mayer', instrument='Guitar', genre='Rock', years_active=27, is_active=True)

In [6]:
try:
    miles_davis = Musician(
        name="Miles Davis",
        instrument="Trumpet",
        genre="Jazz",
        years_active=33,
    )
except ValidationError as e:
    print(f"Error: {e}")

Error: 1 validation error for Musician
instrument
  String should match pattern '(Guitar|Bass|Drums|Vocals|Keyboard)' [type=string_pattern_mismatch, input_value='Trumpet', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/string_pattern_mismatch


## 1.4 Protect fields

In [7]:
# Change an attribute
john_mayer.genre = "Blues"

john_mayer

Musician(name='John Mayer', instrument='Guitar', genre='Blues', years_active=27, is_active=True)

However, if you want to protect it, you can freeze the fields setting the parameter `frozen=True`:

In [8]:
from pydantic import ValidationError

class Musician(BaseModel):
    """Class representing a musician with validated fields."""

    name: StrictStr = Field(
        pattern=r"[A-Z]{1}[a-z]+ [A-Z]{1}[a-z]",
        description="The name of the musician. First and last name, capitalized.",
        frozen=True,
    )
    instrument: StrictStr = Field(
        pattern=r"(Guitar|Bass|Drums|Vocals|Keyboard)",
        description="The instrument played by the musician.",
        frozen=True,
    )
    genre: StrictStr = Field(
        pattern=r"[A-Za-z]+",
        description="The genre of music the musician is associated with.",
        frozen=True,
    )
    years_active: int = Field(
        ge=0,
        le=100,
        description="Number of years the musician has been active in their career.",
    )
    is_active: StrictBool = Field(
        default=True,
        description="Indicates if the musician is currently active.",
    )

john_mayer = Musician(
    name="John Mayer",
    instrument="Guitar",
    genre="Rock",
    years_active=27,
)

In [9]:
try:
    john_mayer.instrument = "Piano"
except ValidationError as e:
    print(f"Error: {e}")

Error: 1 validation error for Musician
instrument
  Field is frozen [type=frozen_field, input_value='Piano', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/frozen_field


# 2. Validation

## 2.1 Field validators

In its simplest form, a field validator is a callable taking the value to be validated as an argument and returning the validated value. The callable can perform checks for specific conditions (see raising validation errors) and make changes to the validated value (coercion or mutation).

Four different types of validators can be used. They can all be defined using the annotated pattern or using the `field_validator()` decorator, applied on a class method:

- __After validators__: run after Pydantic's internal validation. They are generally more type safe and thus easier to implement.

In [10]:
from pydantic import field_validator

class Musician(BaseModel):
    """Class representing a musician with validated fields."""

    name: StrictStr = Field(
        pattern=r"[A-Z]{1}[a-z]+ [A-Z]{1}[a-z]",
        description="The name of the musician. First and last name, capitalized.",
        frozen=True,
    )
    instrument: StrictStr = Field(
        pattern=r"(Guitar|Bass|Drums|Vocals|Keyboard)",
        description="The instrument played by the musician.",
        frozen=True,
    )
    genre: StrictStr = Field(
        pattern=r"[A-Za-z]+",
        description="The genre of music the musician is associated with.",
        frozen=True,
    )
    years_active: int = Field(
        ge=0,
        le=100,
        description="Number of years the musician has been active in their career.",
    )
    is_active: StrictBool = Field(
        default=True,
        description="Indicates if the musician is currently active.",
    )
    list_of_albums: list[StrictStr] = Field(
        default_factory=list,
        description="A list of the musician's albums.",
    )

    @field_validator('list_of_albums', mode='after')
    def check_albums_not_empty(cls, v):
        if not v:
            raise ValueError("The list of albums cannot be empty.")
        return v

In [ ]:
try:
    juan_pablo = Musician(
        name="Juan Pablo",
        instrument="Vocals",
        genre="Reggaeton",
        years_active=2,
        list_of_albums=[],
    )
except ValidationError as e:
    print(f"Error: {e}")

Error: 1 validation error for Musician
list_of_albums
  Value error, The list of albums cannot be empty. [type=value_error, input_value=[], input_type=list]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error


- __Before validators__: run before Pydantic's internal parsing and validation (e.g. coercion of a str to an int). These are more flexible than after validators, but they also have to deal with the raw input, which in theory could be any arbitrary object. You should also avoid mutating the value directly if you are raising a validation error later in your validator function, as the mutated value may be passed to other validators if using unions.
The value returned from this callable is then validated against the provided type annotation by Pydantic.

In [28]:
class Musician(BaseModel):
    """Class representing a musician with validated fields."""

    name: StrictStr = Field(
        pattern=r"[A-Z]{1}[a-z]+ [A-Z]{1}[a-z]",
        description="The name of the musician. First and last name, capitalized.",
        frozen=True,
    )
    instrument: StrictStr = Field(
        pattern=r"(Guitar|Bass|Drums|Vocals|Keyboard)",
        description="The instrument played by the musician.",
        frozen=True,
    )
    genre: StrictStr = Field(
        pattern=r"[A-Za-z]+",
        description="The genre of music the musician is associated with.",
        frozen=True,
    )
    years_active: int = Field(
        ge=0,
        le=100,
        description="Number of years the musician has been active in their career.",
    )
    is_active: StrictBool = Field(
        default=True,
        description="Indicates if the musician is currently active.",
    )
    list_of_albums: list[StrictStr] = Field(
        default_factory=list,
        description="A list of the musician's albums.",
    )

    @field_validator('list_of_albums', mode='before')
    def check_albums_not_empty(cls, v):
        if not v:
            raise ValueError("The list of albums cannot be empty.")
        return v

In [29]:
try:
    juan_pablo = Musician(
        name="Juan Pablo",
        instrument="Vocals",
        years_active=2,
        list_of_albums=[],
    )
except ValidationError as e:
    print(f"Error: {e}")

Error: 2 validation errors for Musician
genre
  Field required [type=missing, input_value={'name': 'Juan Pablo', 'i...2, 'list_of_albums': []}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
list_of_albums
  Value error, The list of albums cannot be empty. [type=value_error, input_value=[], input_type=list]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error


## 2.2 Model validators

Validation can also be performed on the entire model's data using the `model_validator()` decorator.

Three different types of model validators can be used:

- __After validators__: run after the whole model has been validated. As such, they are defined as instance methods and can be seen as post-initialization hooks. Important note: the validated instance should be returned.

In [30]:
from typing_extensions import Self

from pydantic import BaseModel, model_validator


class UserModel(BaseModel):
    username: str
    password: str
    password_repeat: str

    @model_validator(mode='after')
    def check_passwords_match(self) -> Self:
        if self.password != self.password_repeat:
            raise ValueError('Passwords do not match')
        return self

In [31]:
try:
    UserModel(
        username="user1",
        password="securepassword",
        password_repeat="securepassword123",
    )
except ValidationError as e:
    print(f"Error: {e}")

Error: 1 validation error for UserModel
  Value error, Passwords do not match [type=value_error, input_value={'username': 'user1', 'pa...t': 'securepassword123'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error


- __Before validators__: are run before the model is instantiated. These are more flexible than after validators, but they also have to deal with the raw input, which in theory could be any arbitrary object. You should also avoid mutating the value directly if you are raising a validation error later in your validator function, as the mutated value may be passed to other validators if using unions.

In [32]:
from typing import Any

from pydantic import BaseModel, model_validator


class UserModel(BaseModel):
    username: str

    @model_validator(mode='before')
    @classmethod
    def check_card_number_not_present(cls, data: Any) -> Any:  
        if isinstance(data, dict):  
            if 'card_number' in data:
                raise ValueError("'card_number' should not be included")
        return data

In [33]:
try:
    UserModel(
        username="user2",
        card_number="1234-5678-9012-3456",
    )
except ValidationError as e:
    print(f"Error: {e}")

Error: 1 validation error for UserModel
  Value error, 'card_number' should not be included [type=value_error, input_value={'username': 'user2', 'ca...: '1234-5678-9012-3456'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error


__Ordering of validators__:

When using the annotated pattern, the order in which validators are applied is defined as follows: before and wrap validators are run from right to left, and after validators are then run from left to right.
Internally, validators defined using the decorator are converted to their annotated form counterpart and added last after the existing metadata for the field. This means that the same ordering logic applies.